# IBM Watson Library

Full details and documentation on the IBM Watson library, check it ou here: https://pypi.org/project/ibm-watson/

In [ ]:
#  pip install --upgrade "ibm-watson>=5.0.0"

In [ ]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

import json

# Visual Recognition

### Authenticate and Set Service URL

In [ ]:
authenticator = IAMAuthenticator('{APIKEY}')
authenticator

In [ ]:
vis_recog = VisualRecognitionV3(
    version = '2018-03-19',
    authenticator = authenticator
)
vis_recog

In [ ]:
vis_recog.set_service_url('{SERVICE_URL}')

### Set image

In [ ]:
img_url = 'https://thefunnybeaver.com/wp-content/uploads/2017/07/angry-otter.jpg'

### Call service

In [ ]:
classes_result = vis_recog.classify(url = img_url).get_result()

In [ ]:
classes_result

In [ ]:
print(json.dumps(classes_result, indent=2))

### Quick function

In [ ]:
def nice_classify(url):
    classes_result = vis_recog.classify(url = url).get_result()
    print(json.dumps(classes_result, indent=2))

## Built-In Classifiers

https://cloud.ibm.com/apidocs/visual-recognition/visual-recognition-v3?code=python#classify <br>
Check for explicit?!?!

In [ ]:
nice_classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.ytimg.com%2Fvi%2FqRPC6vOrIzE%2Fmaxresdefault.jpg&f=1&nofb=1')

In [ ]:
def nice_classify(url, classifier = ['default']):
    classes_result = vis_recog.classify(
        url = url,
        classifier_ids = classifier
    ).get_result()
    print(json.dumps(classes_result, indent=2))

In [ ]:
nice_classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.ytimg.com%2Fvi%2FqRPC6vOrIzE%2Fmaxresdefault.jpg&f=1&nofb=1', classifier = ['default', 'food', 'explicit'])

## Custom Classifier

Full details here: https://cloud.ibm.com/apidocs/visual-recognition/visual-recognition-v3?code=python#createclassifier

### Create

In [ ]:
with open('./Mantas.zip', 'rb') as mantas, open('./Stingrays.zip', 'rb') as stingrays, open('./Otters.zip', 'rb') as otters:
    model = vis_recog.create_classifier(
        name = 'Batoidea',
        positive_examples = {'manta_ray': mantas, 'stingray': stingrays},
        negative_examples = otters
    ).get_result()

In [ ]:
print(json.dumps(model, indent=2))

There are also APIs that support:
- retrieving a list of available classifiers
- retrieving classifier details
- updating classifieres
- deleting classifiers

### Use

In [ ]:
nice_classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmedia.buzzle.com%2Fmedia%2Fimages-en%2Fgallery%2Ffish%2F1200-55844925-stingray-grand-cayman-island.jpg&f=1&nofb=1', ['default', 'Batoidea_550385256'])

# Text to Speech

In [4]:
from ibm_watson import TextToSpeechV1

In [5]:
TTSauthenticator = IAMAuthenticator('jSnMxbNInJ2grxO1hUifi7JQ7Ivfc4Qccji0fXC08cJD')
text_to_speech = TextToSpeechV1(
    authenticator = TTSauthenticator
)

text_to_speech.set_service_url('https://gateway-lon.watsonplatform.net/text-to-speech/api')

## List Voices

In [8]:
voices = text_to_speech.list_voices().get_result()

In [9]:
print(json.dumps(voices, indent=2))

{
  "voices": [
    {
      "gender": "female",
      "supported_features": {
        "custom_pronunciation": true,
        "voice_transformation": false
      },
      "name": "pt-BR_IsabelaV3Voice",
      "customizable": true,
      "description": "Isabela: Brazilian Portuguese (portugu\u00eas brasileiro) female voice. Dnn technology.",
      "language": "pt-BR",
      "url": "https://api.eu-gb.text-to-speech.watson.cloud.ibm.com/instances/e25f3e48-3f48-48ea-9e5d-7e9dfe326539/v1/voices/pt-BR_IsabelaV3Voice"
    },
    {
      "gender": "male",
      "supported_features": {
        "custom_pronunciation": true,
        "voice_transformation": false
      },
      "name": "de-DE_DieterV2Voice",
      "customizable": true,
      "description": "Dieter: Standard German (Standarddeutsch) male voice. Dnn technology.",
      "language": "de-DE",
      "url": "https://api.eu-gb.text-to-speech.watson.cloud.ibm.com/instances/e25f3e48-3f48-48ea-9e5d-7e9dfe326539/v1/voices/de-DE_DieterV2Voice"
 

## Synthesise Voice

In [36]:
synthesis = text_to_speech.synthesize(
    text = 'Buenos dias amigos. Como estas?',
    voice = 'es-ES_EnriqueV3Voice',
    accept='audio/wav'
).get_result()

In [37]:
synthesis

<Response [200]>

In [35]:
with open('Test.wav', 'wb') as audio_test:
    audio_test.write(synthesis.content)